In [60]:
# importing libraries
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
# reading data with house pricing
train = pd.read_csv('Data/train.csv', index_col='Id')
test = pd.read_csv('Data/test.csv', index_col='Id') # used at the end to evaluation model efficiency

In [30]:
# examining data
# print('SalePrice' in train.columns, 'SalePrice' in test.columns, sep='\n')
# train.head()

In [74]:
# setting 'Price' column as target for predctions
y = train.SalePrice

#features preparation
X = train.drop(['SalePrice'], axis=1) # remove 'SalePrice' column

# only numerical predicators from test dataset
X_test_full = test.select_dtypes(exclude=['object'])

# divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [75]:
# retrieves categorical columns from features dataset
cat_cols = [col for col in X_train_full.columns if 
                    X_train_full[col].dtype == 'object']

# creates train and valid datasets with categorical data only
X_train_cat = X_train_full[cat_cols]
X_valid_cat = X_valid_full[cat_cols]

In [66]:
# prepares features to imputation
# drop categorical columns
X_train_full.drop(cat_cols, axis=1, inplace=True)
X_valid_full.drop(cat_cols, axis=1, inplace=True)

In [67]:
# define function for column imputation (missing values)
def imputation_dataset(X_train, X_valid, X_test):
    imputer = SimpleImputer()
    
    imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
    imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))
    imputed_X_test = pd.DataFrame(imputer.transform(X_test))
    
    imputed_X_train.columns = X_train.columns
    imputed_X_valid.columns = X_valid.columns
    imputed_X_test.columns = X_test.columns
    
    return [imputed_X_train, imputed_X_valid, imputed_X_test]

In [68]:
# imputing columns with missing values
imp_X_train, imp_X_valid, imp_X_test = imputation_dataset(X_train_full, X_valid_full, X_test_full)

In [76]:
X_train_cat.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,New,Partial
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,PosN,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [96]:
# retrieves categorical columns with missing values
train_cat_cols_missing = [col_t for col_t in X_train_cat.columns if 
                 X_train_cat[col_t].isnull().any()]

valid_cat_cols_missing = [col_v for col_v in X_valid_cat.columns if 
                 X_valid_cat[col_v].isnull().any()]

# print(len(train_cat_cols_missing), len(valid_cat_cols_missing), sep='\n')

t_cat_miss = train_cat_cols_missing.copy()

for t in train_cat_cols_missing:
    for v in valid_cat_cols_missing:
        if t == v:
            del t_cat_miss[t_cat_miss.index(t)]

print(t_cat_miss)

# creates dataset of categorical columns without missing values
# X_cat_full = X_cat[cat_cols_full]

['Electrical']


In [56]:
# prepares dataset of categorical columns to concatenation
# reset index
X_cat_full.reset_index(drop=True)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,New,Partial
1,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,PosN,Norm,...,TA,CBlock,GasA,Gd,N,TA,Typ,Y,WD,Normal
2,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,...,Gd,BrkTil,GasA,Ex,Y,TA,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
4,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1164,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,...,TA,CBlock,GasA,TA,Y,Gd,Min2,Y,WD,Normal
1165,RM,Pave,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,RRAe,Norm,...,TA,Slab,GasA,TA,Y,TA,Typ,Y,WD,Normal
1166,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal


In [21]:


train[cat_cols].iloc[:imp_X_train.shape[0]]

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
2,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,CBlock,GasA,Ex,Y,TA,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
4,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,TA,BrkTil,GasA,Gd,Y,Gd,Typ,Y,WD,Abnorml
5,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Feedr,Norm,...,TA,CBlock,GasA,TA,Y,TA,Typ,Y,WD,Alloca
1165,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1166,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,New,Partial


In [15]:
categorical_features = train[cat_cols].iloc[:imp_X_train.shape[0]]
# categorical_features.shape
X_train_proc = pd.concat([imp_X_train, categorical_features], axis=1)

In [16]:
X_train_proc.shape

(1169, 63)

In [17]:
# evaluating which columns has missing values (only from numerical features)
cols_with_missing = [col for col in X_train_proc if X_train_proc[col].isnull().any()]
X_train_proc[cols_with_missing]


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,20.0,90.000000,11694.0,9.0,5.0,2007.0,2007.0,452.0,48.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20.0,60.000000,6600.0,5.0,5.0,1962.0,1962.0,0.0,0.0,0.0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
2,30.0,80.000000,13360.0,5.0,7.0,1921.0,2006.0,0.0,713.0,0.0,...,TA,CBlock,GasA,Ex,Y,TA,Typ,Y,WD,Normal
3,20.0,69.614017,13265.0,8.0,5.0,2002.0,2002.0,148.0,1218.0,0.0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
4,20.0,118.000000,13704.0,7.0,5.0,2001.0,2002.0,150.0,0.0,0.0,...,TA,BrkTil,GasA,Gd,Y,Gd,Typ,Y,WD,Abnorml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,20.0,60.000000,9600.0,4.0,7.0,1950.0,1995.0,0.0,442.0,0.0,...,TA,CBlock,GasA,TA,Y,TA,Typ,Y,WD,Alloca
1165,90.0,68.000000,8930.0,6.0,5.0,1978.0,1978.0,0.0,0.0,0.0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal
1166,120.0,69.614017,3196.0,7.0,5.0,2003.0,2004.0,18.0,0.0,0.0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,New,Partial
1167,60.0,58.000000,16770.0,7.0,5.0,1998.0,1998.0,30.0,0.0,0.0,...,TA,PConc,GasA,Ex,Y,Gd,Typ,Y,WD,Normal


In [13]:
# drop columns with missing values from training and test datasets
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

In [22]:
X_train_full.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


### Evaluating cardinality in categorical values

In [17]:
# retrieving columns with low cardinality
low_cardinality_cols = [col for col in X_train_full.columns if 
                        X_train_full[col].nunique() < 10 and
                        X_train_full[col].dtype == "object"]
low_cardinality_cols

[]

In [18]:
#
num_cols = [col for col in X_train_full if X_train_full[col].dtype in 
            ['int64', 'float64']]
num_cols

9

In [19]:
correct_cols = low_cardinality_cols + num_cols
correct_cols

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']